In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop,Adam
import numpy as np
import pandas as pd
#from tragedy import team
import itertools
from random import choice, randint
from keras.models import load_model



In [3]:
dat2 = pd.read_csv('outfit_w_color_pattern.csv')
## build vocab
def convert_agent_output_to_judge_input(input_str):
    broken_out = input_str.replace('_',' ')
    input_str = broken_out +' '+input_str
    return input_str
convert_agent_output_to_judge_input('top_melange_grey')

dat2['combined_tax_att'] = ''
for index,row in dat2.iterrows():
    tax_cat = row['taxonomy_cat']
    pattern = row['pattern_tonks']
    color = row['color_tonks']
    
    combined_cat = tax_cat+'_'+str(pattern)+'_'+str(color)
    combined_cat = combined_cat.replace('[','')
    combined_cat = combined_cat.replace(']','')
    combined_cat = combined_cat.replace("'",'')
    
    dat2.loc[index,'combined_tax_att'] = combined_cat
dat2['for_judge'] = ''
for index, row in dat2.iterrows():
    tax_att = row['combined_tax_att']
    dat2.loc[index,'for_judge'] = convert_agent_output_to_judge_input(tax_att)
    
    

# use_predicted_probability refactor

In [4]:
dat2.head()

,Unnamed: 0,Unnamed: 0.1,original_image,cropped_img_name,taxonomy_cat,color_tonks,color_prob,pattern_tonks,pattern_prob,combined_tax_att,for_judge
0,0,0,fall_outfit_flatlay_00000000.jpg,fall_outfit_flatlay_00000000_2.jpg,bottom,navy,0.999985,melange,0.999436,bottom_melange_navy,bottom melange navy bottom_melange_navy
1,1,1,fall_outfit_flatlay_00000001.jpg,fall_outfit_flatlay_00000001_3.jpg,bottom,grey,0.999694,plaid,0.934133,bottom_plaid_grey,bottom plaid grey bottom_plaid_grey
2,2,2,fall_outfit_flatlay_00000006.jpg,fall_outfit_flatlay_00000006_2.jpg,bottom,blue,0.967016,melange,0.926557,bottom_melange_blue,bottom melange blue bottom_melange_blue
3,3,3,fall_outfit_flatlay_00000007.jpg,fall_outfit_flatlay_00000007_2.jpg,bottom,navy,0.999993,melange,0.999404,bottom_melange_navy,bottom melange navy bottom_melange_navy
4,4,4,fall_outfit_flatlay_00000008.jpg,fall_outfit_flatlay_00000008_3.jpg,bottom,blue,0.999270,melange,0.998162,bottom_melange_blue,bottom melange blue bottom_melange_blue


In [6]:
combined_text_list = list(dat2['taxonomy_cat'].unique())

In [7]:
tax_att_dict = {}
index = 0
#tax_att_dict[0] = 'skip'
for i in combined_text_list:
    tax_att_dict[index] = i
    index+=1
    

In [8]:
tax_att_dict[3]

'shoes'

In [10]:
'''
tax_att_dict = {0:'dress blue dotted',
                1:'top solid white',
               2: 'bottom solid pink',
               3:'shoes solid green',
               4: 'outerwear blue melange',
               5: 'top cheetah beige'}
               
'''

In [9]:
def use_predicted_probability(label_dict,pred_key):
    return label_dict[pred_key]

use_predicted_probability(tax_att_dict,4)

'top'

In [10]:
len(tax_att_dict)

5

In [11]:
## random action refactor
def random_outfit(outfit_dict):
    outfit_to_play = randint(0,(len(outfit_dict)-1))
    return outfit_to_play


use_predicted_probability(tax_att_dict,random_outfit(tax_att_dict))

'bottom'

In [12]:
for i in range(20):
    zz = random_outfit(tax_att_dict)
    print(zz, use_predicted_probability(tax_att_dict,zz))

3 shoes
4 top
3 shoes
3 shoes
0 bottom
2 outerwear
4 top
1 dress
4 top
3 shoes
3 shoes
2 outerwear
3 shoes
2 outerwear
3 shoes
0 bottom
0 bottom
1 dress
1 dress
3 shoes


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = dat2['taxonomy_cat'].tolist()

vectorizer = CountVectorizer()
vectorizer = vectorizer.fit(corpus)
len(vectorizer.get_feature_names())

#update(new_state,reward)      

5

In [14]:
#zz = use_predicted_probability(tax_att_dict,random_outfit(tax_att_dict))
zz = 'top_solid_brown top_solid_brown top_solid_brown top_melange_grey'
print(zz)
print(convert_agent_output_to_judge_input(zz))

vectorized_otfit = vectorizer.transform([convert_agent_output_to_judge_input(zz)])


top_solid_brown top_solid_brown top_solid_brown top_melange_grey
top solid brown top solid brown top solid brown top melange grey top_solid_brown top_solid_brown top_solid_brown top_melange_grey


In [15]:
zz = vectorized_otfit.toarray()

In [16]:
len(list(zz[0]))

5

In [17]:
zz[0]

array([0, 0, 0, 0, 4])

In [24]:
# Outfiting env
from random import choice, randint

#play tragedy of the commons
import random
import math
#rl_model_name = 'altruistic_1_9_26'
rl_model_name = 'greed_1_9_26'

class outfit_bot:
    def __init__(self,type=None,action_dict=None):
        self.outfit_score = 0
        self.type = type
        self.label_dict=action_dict
    def get_action(self, state =None):
        return self.label_dict[randint(0,(len(self.label_dict)-1))]

        
    



class TOC:
    def __init__(self, num_learning_rounds =None, learner = None, report_every=1000, vectorizer=None,outfit_bot=None):
        self._num_learning_rounds = num_learning_rounds
        self._report_every = report_every
        self.player = outfit_bot #change to learner later
        self.game = 1
        self.evaluation = False
        self.outfit_records = []
        
    def play_game(self):
        
        p1,initial_item,outfit_score = self.reset_game()
        #opp  last  played, player last played, opp_score,player_score,turn#, current_fishery_count, 
        state = initial_item
        turn_count = 0
        print(state,outfit_score)
        pre_outfit_score = outfit_score
        #every "game" is going to be some number of rounds where it picks an item at every round
        # for now setting it to 4
        for turn in range(4):
            vectorized_state = vectorizer.transform([convert_agent_output_to_judge_input(state)])
            vectorized_state = list(vectorized_state.toarray()[0])
            
            next_clothing_choice = p1.get_action(vectorized_state)
            if 'bottom' in state:
                round_outfit_score = state.count('bottom')*10
            else:
                round_outfit_score = 1
            round_score_diff = round_outfit_score - pre_outfit_score
            pre_outfit_score = round_outfit_score
            
            print(next_clothing_choice)
            state = state +' '+ next_clothing_choice
            
            print(state, round_score_diff)
            #p1.update(state1,round_outfit_score) #state + reward from the environment


            if  self.evaluation == True:
                print('outfit score :',outfit_score)

        if self.evaluation == False:
            self.game += 1
            self.outfit_records.append(outfit_score)
            self.report()

        if self.game == self._num_learning_rounds:
            print("Turning off learning!")
            self.player._learning = False
            self.win = 0
            self.loss = 0

    def reset_game(self):
        '''
        steps to reset:
        1) get random item out of the product catalog can select a thing out of the label dict
        
        ## random action refactor
        def random_outfit(tax_att_dict):
            outfit_to_play = randint(0,(len(tax_att_dict)-1))
            return outfit_to_play
        
        score set to 0
        '''
        p1  = self.player
        #print(self.player)
        outfit_score = 0 # outfitt score should be sent through model/rules for a 1 outfit item 
    
        #I shpould probably just have a function for this?
        initial_item = self.player.label_dict[random_outfit(self.player.label_dict)]

        return p1,initial_item,outfit_score
    
    def report(self):
        #turned off for plotting 9/18
        if self.game % self._num_learning_rounds == 0:

            avg_outfit_score =  sum(self.outfit_records) / float(len(self.outfit_records))
            self.turn_record = []
            print('##############################################')
            print('#                 Final Score                #')
            print('##############################################')
            print('')
            print(str(self.game))
            print('')
            print('average game length: ',avg_outfit_score)
            print('##############################################')
            
            #turning off this section for testing
            self.evaluation = True
            self.player._epsilon = 1.0
            print('#################### G1 ######################')
            self.play_game()
            print('#################### G2 ######################')
            self.play_game()
            print('#################### G3 ######################')
            self.play_game()
            self.player._epsilon = .9
            self.evaluation = False
            
            #self.player.save_rl_model('models/{}_iteration_{}'.format(rl_model_name,self.game))
            
        elif self.game % self._report_every == 0:
            avg_outfit_score =  sum(self.outfit_records) / float(len(self.outfit_records))
            self.turn_record = []
            print('##############################################')
            print('#                Updated Score               #')
            print('##############################################')
            print('')
            print(str(self.game))
            print('')
            print('average game length: ',avg_outfit_score)
            print('##############################################')
            
            #turning off this section for testing
            self.evaluation = True
            self.player._epsilon = 1.0
            print('#################### G1 ######################')
            self.play_game()
            print('#################### G2 ######################')
            self.play_game()
            print('#################### G3 ######################')
            self.play_game()
            self.player._epsilon = .9
            self.evaluation = False
            
            #self.player.save_rl_model('models/{}_iteration_{}'.format(rl_model_name,self.game))

            
bot1 = outfit_bot(action_dict = tax_att_dict)

game1 = TOC(num_learning_rounds = 1, vectorizer = vectorizer, outfit_bot = bot1)
#game1.play_game()

In [26]:
game1.play_game()

shoes 0
bottom
shoes bottom 1
outerwear
shoes bottom outerwear 9
outerwear
shoes bottom outerwear outerwear 0
shoes
shoes bottom outerwear outerwear shoes 0
##############################################
#                 Final Score                #
##############################################

3

average game length:  0.0
##############################################
#################### G1 ######################
shoes 0
bottom
shoes bottom 1
outfit score : 0
outerwear
shoes bottom outerwear 9
outfit score : 0
dress
shoes bottom outerwear dress 0
outfit score : 0
shoes
shoes bottom outerwear dress shoes 0
outfit score : 0
#################### G2 ######################
shoes 0
shoes
shoes shoes 1
outfit score : 0
shoes
shoes shoes shoes 0
outfit score : 0
bottom
shoes shoes shoes bottom 0
outfit score : 0
bottom
shoes shoes shoes bottom bottom 9
outfit score : 0
#################### G3 ######################
bottom 0
bottom
bottom bottom 10
outfit score : 0
bottom
bottom bottom bot